# Marketing Analytics : Assignment 2 by Kerrian Le Bars

## Demand and Profitability Analysis for a Multi-Product Category 

You are a marketing analytic consultant doing category analysis for a multi-product category. You are asked to perform pricing and profitability analysis for this category. The market has three products sold by different firms: 1, 2, and 3. You have been informed that the marginal cost of the products are C1 = $0.5, C2 = $0.75, C3 = $0.9, and the retail margin for all the products is 10%. We denote the prices of the three products as P1, P2 and P3. 

Based on the demand models that your data science team has run on the sales and pricing data for this category, you know that the own and cross price elasticities for the three products can be specified using the following demand system :

$Q_{1} = A_{1} \cdot P_{1}^{-2.5} \cdot P_{2}^{1.5} \cdot P_{3}^{1.2}$

$Q_{2} = A_{2} \cdot P_{1}^{0.7} \cdot P_{2}^{-1.5} \cdot P_{3}^{0.5}$

$Q_{3} = A_{3} \cdot P_{1}^{0.7} \cdot P_{2}^{0.4} \cdot P_{3}^{-1.2}$

Further, the base prices of three products are given as : P1 = $2, P2 = $1.75 and P3 = $1.5

In [1]:
# Import the numpy library for numerical operations, including arrays and mathematical functions
import numpy as np

# Import the pandas library for data manipulation and analysis, particularly for handling dataframes
import pandas as pd

# Import the symbols, Eq, and solve functions from the sympy library for symbolic mathematics
from sympy import symbols, Eq, solve

### Question 1

***What are the own price elasticites of the three products?***

The demand for product i in the market with k product is :

$Q_{i} = A_{i} \cdot P_{1}^{b_{i,1}} \cdot P_{2}^{b_{i,2}} ... \cdot P_{k}^{b_{i,k}}$

The coefficient on the price of product i in the demand equation for product i is the own price elasticity of product i :

$b_{i,i} = \frac{\partial Q_{i}}{\partial P_{i}} \cdot \frac{P_{i}}{Q_{i}}$

We deduce that the own price elasticity for :
- the product 1 is $b_{1,1}$ that equal to -2.5
- the product 2 is $b_{2,2}$ that equal to -1.5
- the product 3 is $b_{3,3}$ that equal to -1.2

### Question 2

***Write down a matrix of price elasticities***

In [2]:
def price_elasticity_matrix(elasticities):
    """
    Creates a DataFrame representing the price elasticity matrix for a given set of products.
    
    Parameters:
    - elasticities: A 2D list or array where each entry elasticities[i][j] represents the 
                    elasticity of quantity demanded for product i with respect to the price of product j.
                    
    Returns:
    - A pandas DataFrame where rows represent products' quantities (Q1, Q2, ..., Qn) and 
      columns represent products' prices (P1, P2, ..., Pn), allowing for clear labeling of the elasticity matrix.
    """
    
    # Step 1: Determine the number of products based on the length of the elasticity matrix.
    num_products = len(elasticities)
    
    # Step 2: Create row labels (index) for the DataFrame.
    # Each row label corresponds to the quantity of a product, labeled as Q1, Q2, ..., Qn.
    index_labels = [f"Q{i+1}" for i in range(num_products)]
    
    # Step 3: Create column labels for the DataFrame.
    # Each column label corresponds to the price of a product, labeled as P1, P2, ..., Pn.
    column_labels = [f"P{i+1}" for i in range(num_products)]
    
    # Step 4: Create a pandas DataFrame with the given elasticity values.
    # - `elasticities` is the data for the matrix.
    # - `index=index_labels` sets the row labels to Q1, Q2, ..., Qn.
    # - `columns=column_labels` sets the column labels to P1, P2, ..., Pn.
    elasticity_df = pd.DataFrame(
        elasticities,
        index=index_labels,
        columns=column_labels
    )
    
    # Step 5: Return the DataFrame, which represents the elasticity matrix with clear labeling.
    return elasticity_df

In [3]:
# Step 1: Define the elasticity matrix as a 2D list.
# This matrix represents how the quantity demanded of each product (rows) responds to changes
# in the price of each product (columns).
elasticities = [
    [-2.5, 1.5, 1.2],   # Elasticities for the first product with respect to P1, P2, and P3
    [0.7, -1.5, 0.5],    # Elasticities for the second product with respect to P1, P2, and P3
    [0.7, 0.4, -1.2]     # Elasticities for the third product with respect to P1, P2, and P3
]

# Step 2: Create the elasticity matrix DataFrame using the function defined earlier.
# `price_elasticity_matrix(elasticities)` converts the list of lists into a labeled DataFrame,
# making it easier to interpret the elasticity relationships between products.
elasticity_matrix = price_elasticity_matrix(elasticities)

# Step 3: Display the elasticity matrix.
# This shows the DataFrame with labels for quantities (rows) and prices (columns),
# providing a clear, tabular view of cross-product elasticity values.
print(elasticity_matrix)

     P1   P2   P3
Q1 -2.5  1.5  1.2
Q2  0.7 -1.5  0.5
Q3  0.7  0.4 -1.2


Using the fact that $b_{i,j}$ corresponds to the quantity of product i with respect to the price of j, we have that the matrix of price elasticities is :

$\begin{bmatrix}
b_{1,1} & b_{1,2} & b_{1,3} \\
b_{2,1} & b_{2,2} & b_{2,3} \\
b_{3,1} & b_{3,2} & b_{3,3}
\end{bmatrix}$ =
$\begin{bmatrix}
-2.5 & 1.5 & 1.2 \\
0.7 & -1.5 & 0.5 \\
0.7 & 0.4 & -1.2
\end{bmatrix}$

As we can see, we get the same matrix as the one produced by our function "price_elasticity_matrix".

### Question 3

Now assume that product 1 increases its price by $γ_{1} = 0.05$, product 2 keeps its price constant ($γ_{2} = 0$) and product 3 reduces its price by $γ_{3} = -0.15$. Calculate what happens to the quantities of the three products

#### Question A

***Calculate (using a calculator) the ratio of new quantity to the old quantity for each of the three products : $\frac{Q_{1,1}}{Q_{1,0}}$, $\frac{Q_{2,1}}{Q_{2,0}}$ and $\frac{Q_{3,1}}{Q_{3,0}}$. Please show the formulae used and the steps in your calculation.***

The ratio of unit sales of product i after versus before the price change is :

$\frac{Q_{i,1}}{Q_{i,0}} = (1+γ_{1})^{b_{i,1}} \cdot (1+γ_{2})^{b_{i,2}} ... \cdot (1+γ_{k})^{b_{i,k}}$

So we can compute the quantity ratio of each of the three products :

- For the product 1, the ratio between the quantity after the price change and before the price change is :

    ⇔  $\frac{Q_{1,1}}{Q_{1,0}} = (1+γ_{1})^{b_{1,1}} \cdot (1+γ_{2})^{b_{1,2}} \cdot (1+γ_{3})^{b_{1,3}}$

    ⇔  $\frac{Q_{1,1}}{Q_{1,0}} = (1+00.5)^{-2.5} \cdot (1+0)^{1.5} \cdot (1-0.15)^{1.2}$

    ⇔  $\frac{Q_{1,1}}{Q_{1,0}} ≈ 0.885 \cdot 1 \cdot 0.823$

    ⇔  $\frac{Q_{1,1}}{Q_{1,0}} ≈ 0.728$

- For the product 2, the ratio between the quantity after the price change and before the price change is :

    ⇔  $\frac{Q_{2,1}}{Q_{2,0}} = (1+γ_{1})^{b_{2,1}} \cdot (1+γ_{2})^{b_{2,2}} \cdot (1+γ_{3})^{b_{2,3}}$

    ⇔  $\frac{Q_{2,1}}{Q_{2,0}} = (1+00.5)^{0.7} \cdot (1+0)^{-1.5} \cdot (1-0.15)^{0.5}$

    ⇔  $\frac{Q_{2,1}}{Q_{2,0}} ≈ 1.035 \cdot 1 \cdot 0.922$

    ⇔  $\frac{Q_{2,1}}{Q_{2,0}} ≈ 0.954$

- For the product 2, the ratio between the quantity after the price change and before the price change is :

    ⇔  $\frac{Q_{3,1}}{Q_{3,0}} = (1+γ_{1})^{b_{3,1}} \cdot (1+γ_{2})^{b_{3,2}} \cdot (1+γ_{3})^{b_{3,3}}$

    ⇔  $\frac{Q_{3,1}}{Q_{3,0}} = (1+00.5)^{0.7} \cdot (1+0)^{0.4} \cdot (1-0.15)^{-1.2}$

    ⇔  $\frac{Q_{3,1}}{Q_{3,0}} ≈ 1.035 \cdot 1 \cdot 1.215$

    ⇔  $\frac{Q_{3,1}}{Q_{3,0}} ≈ 1.258$

#### Question B

***Interpret and discuss why each of these changes is happening. Which products see an increase in demand and which see a decrease (and why)?***

Product 1 : Price Increase

The ratio of the quantity after the price change to the quantity before the price change is $\frac{Q_{1,1}}{Q_{1,0}} ≈ 0.728$

- The price of Product 1 increases by 5% ($γ_{1} = 0.05$)
- The exponent for Product 1 with respect to its own price is $b_{1,1} = -2.5$, which is highly negative. This means that Product 1 has a very elastic demand with respect to its own price: a small price increase leads to a large reduction in quantity demanded.
- Product 2’s price does not change ($γ_{2} = 0$), so it has no effect on Product 1’s demand.
- Product 3’s price decreases by 15% ($γ_{3} = -0.15$) and the elasticity for Product 1 with respect to Product 3 is $b_{1,3} = 1.2$, which implies that the demand for Product 1 slightly increases due to the decrease in Product 3's price, but this effect is much weaker than the negative effect of the price increase for Product 1 itself.

Therefore, the overall effect is a decrease in the quantity demanded for Product 1 by approximately 27%.

Product 2 : Price Constant

The ratio of the quantity after the price change to the quantity before the price change is $\frac{Q_{2,1}}{Q_{2,0}} ≈ 0.954$

- The price of Product 1 increases by 5% ($γ_{1} = 0.05$)
- The exponent for Product 2 with respect to Product 1 is $b_{2,1} = 0.7$, which is positive. This means that Product 2 has a substitute relationship with Product 1: as the price of Product 1 increases, the demand for Product 2 increases.
- The price of Product 2 itself does not change ($γ_{2} = 0$), so there’s no direct effect on its own demand.
- The price of Product 3 decreases by 15% ($γ_{3} = -0.15$) and the exponent for Product 2 with respect to Product 3 is $b_{2,3} = 0.5$, indicating that Product 2 is a substitute to Product 3 as well. Therefore, the decrease in Product 3’s price reduces the demand for Product 2 and this effect is stronger than the positive effect from the price increase of Product 1.

Thus, the overall demand for Product 2 decreases by approximately 4.6%.

Product 3 : Price Decrease

The ratio of the quantity after the price change to the quantity before the price change is $\frac{Q_{3,1}}{Q_{3,0}} ≈ 1.258$

- The price of Product 1 increases by 5% ($γ_{1} = 0.05$)
- The exponent for Product 3 with respect to Product 1 is $b_{3,1} = 0.7$, indicating that an increase in Product 1’s price increases the demand for Product 3.
- Product 2's price does not change ($γ_{2} = 0$), so it has no direct effect on Product 3’s demand.
- The price of Product 3 decreases by 15% ($γ_{3} = -0.15$) and the exponent for Product 3 with respect to its own price is $b_{3,3} = -1.2$. This strong negative elasticity with respect to its own price means that a decrease in the price of Product 3 results in a significant increase in demand for Product 3.

The combination of the price increase of Product 1 and the price reduction of Product 3 causes the demand for Product 3 to increase by approximately 25.8%.

#### Question C

***Write a function that takes as input – (1) the price changes for the three products ($γ_{1}$, $γ_{2}$, $γ_{3}$) and (2) price elasticities and cross price elasticties for all the products, and gives as output the the ratio of the new to old quantities for the three products.***

In [4]:
def quantity_ratio(price_changes, elasticities):
    """
    Calculate the ratio of unit sales after versus before the price change for each product.

    Parameters:
    - price_changes (list of float): The price changes [γ1, γ2, γ3, ...]. 
      This list represents the percentage change in price for each product.
    - elasticities (list of list of float): An NxN list where each sub-list contains
      the elasticities for a product with respect to each product's price.
      The elasticities matrix describes how the quantity demanded of each product responds 
      to price changes of all products.

    Returns:
    - dict: A dictionary containing the quantity ratios for each product, labeled 
      as 'quantity_ratio_1', 'quantity_ratio_2', etc.
      The value for each product is the ratio of quantity sold after the price change 
      relative to the quantity before the price change.
    """
    
    # Step 1: Initialize a dictionary to store the calculated quantity ratios.
    # The dictionary will map keys like 'quantity_ratio_1', 'quantity_ratio_2', etc., 
    # to the respective quantity ratio for each product.
    quantity_ratios = {}

    # Step 2: Determine the number of products based on the size of the elasticities matrix.
    # The number of products is equal to the number of rows (or columns) in the elasticities matrix.
    num_products = len(elasticities)
    
    # Step 3: Loop through each product to calculate its quantity ratio.
    for i in range(num_products):
        # Initialize the quantity ratio for product i to 1 (the base case before any price change).
        ratio = 1
        
        # Step 4: Loop through each price change for all products (j = 0 to num_products-1).
        # This nested loop accounts for the effect of each product’s price change on the 
        # quantity demanded of product i, according to the elasticities.
        for j in range(num_products):
            # Update the quantity ratio by multiplying with the appropriate elasticity factor.
            # The term (1 + price_changes[j]) accounts for the price change effect on product j,
            # and this is raised to the power of the elasticity between product i and product j.
            ratio *= (1 + price_changes[j]) ** elasticities[i][j]
        
        # Step 5: Store the calculated ratio for product i in the dictionary with a corresponding label.
        # The key is a string such as 'quantity_ratio_product_1', 'quantity_ratio_product_2', etc.
        quantity_ratios[f'quantity_ratio_product_{i+1}'] = ratio
    
    # Step 6: Return the dictionary containing the quantity ratios for each product.
    return quantity_ratios

#### Question D

***Evaluate the function at the values of γ1, γ2, γ3 specified earlier and store the results in an array named "quantity_changes". Check if the results from the function are the same as that in Question A.***

In [5]:
# Step 1: Define the price changes as a list.
# `price_changes` represents the percentage change in prices for each product.
# Here, product 1 has a price increase of 5% (0.05), product 2 has no change in price (0), 
# and product 3 has a price decrease of 15% (-0.15).
price_changes = [0.05, 0, -0.15]

# Step 2: Define the elasticity matrix as a 2D list.
# This matrix represents the elasticity of quantity demanded for each product (rows)
# with respect to the price of each product (columns), similar to before.
elasticities = [
    [-2.5, 1.5, 1.2],   # Elasticities for the first product with respect to P1, P2, and P3
    [0.7, -1.5, 0.5],    # Elasticities for the second product with respect to P1, P2, and P3
    [0.7, 0.4, -1.2]     # Elasticities for the third product with respect to P1, P2, and P3
]

# Step 3: Call the `quantity_ratio` function with the price changes and elasticities.
# This function calculates how the quantity demanded for each product changes in response 
# to the specified price changes, taking into account cross-product elasticities.
quantity_ratios = quantity_ratio(price_changes, elasticities)

# Step 4: Print the `quantity_ratios` result.
# This output shows the calculated quantity ratios for each product after applying the 
# price changes, reflecting the percentage change in quantity demanded for each product.
print(quantity_ratios)

# Step 5: Print the type of `quantity_ratios`.
# This line verifies that the result is a specific data type (e.g., list, numpy array).
print(type(quantity_ratios))

{'quantity_ratio_product_1': 0.7283321201492527, 'quantity_ratio_product_2': 0.95398593595287, 'quantity_ratio_product_3': 1.2575631372951144}
<class 'dict'>


To make it easier to read the results of the changes in quantities of the different products, we decided to store them in a dictionary. However, for data manipulation reasons we will transform them into a "np.array" format.

In [6]:
# Step 1: Convert the `quantity_ratios` values into a NumPy array.
# `quantity_ratios` is assumed to be a dictionary with values representing quantity changes
# for each product. The `list(quantity_ratios.values())` extracts these values as a list.
# Wrapping this in `np.array()` converts the list into a NumPy array.
quantity_changes = np.array(list(quantity_ratios.values()))

# Step 2: Print the `quantity_changes` array.
# This will output the array containing quantity change values, making it easy to see 
# the percentage or ratio adjustments for each product's quantity demanded.
print(quantity_changes)

# Step 3: Print the type of `quantity_changes`.
# This confirms that `quantity_changes` is a NumPy array.
# Expected output: <class 'numpy.ndarray'>
print(type(quantity_changes))

[0.72833212 0.95398594 1.25756314]
<class 'numpy.ndarray'>


To more easily (and scientifically) compare the results of our function and those performed with the calculator, we decided to create a function capable of calculating the difference in errors in percentage between two arrays.

In [7]:
def calculate_percentage_errors(array1, array2, absolute=True):
    """
    Calculate the percentage errors between two arrays.

    Parameters:
    - array1 (list of float): The first array containing the true or original values.
    - array2 (list of float): The second array containing the predicted or observed values.
    - absolute (bool, optional): If True, calculates the absolute percentage error. 
      If False, calculates the signed percentage error. Default is True.

    Returns:
    - list of float: A list of percentage errors between the elements of `array1` and `array2`.
    """
    # Step 1: Check if both arrays have the same length
    if len(array1) != len(array2):
        raise ValueError("Both arrays must have the same length.")
    
    # Step 2: Initialize an empty list to store the percentage errors
    errors = []
    
    # Step 3: Loop through corresponding elements of array1 and array2
    for a, b in zip(array1, array2):
        # Step 4: Ensure that no element in array1 is zero to avoid division by zero
        if a == 0:
            raise ValueError("Values in array1 must be non-zero to calculate percentage errors.")
        
        # Step 5: Calculate the percentage error
        # If `absolute` is True, calculate the absolute percentage error
        if absolute:
            error = abs(a - b) / abs(a) * 100
        else:
            # If `absolute` is False, calculate the signed percentage error
            error = ((a - b) / a * 100)
        
        # Step 6: Append the calculated error to the errors list
        errors.append(error)
    
    # Step 7: Return the list of errors
    return errors

In [8]:
# Assign the qunatity changes calculated using elasticities to a variable
quantity_changes_python = quantity_changes

# Define a numpy array for the actual quantity changes (calculated by hand)
quantity_changes_calculator = np.array([0.728, 0.954, 1.258])

# Calculate the percentage errors between the two profit change calculations
errors = calculate_percentage_errors(quantity_changes_python, quantity_changes_calculator)

# Output the calculated errors as a list of percentage differences
print(errors)

[0.04560009644840444, 0.00147424051026023, 0.03473882876570935]


As we can see the difference in errors between the two methods for :

- the product 1 is approximatively 0.05%
- the product 2 is approximatively 0.001%
- the product 3 is approximatively 0.03%

We can therefore conclude with confidence that the results between the two methods are substantially identical. The function gives us the same results as the calculations.

### Question 4

Using the same price changes as in Question 3, calculate what happens to the profits of the three products.

#### Question A

***Calculate (using a calculator) the ratio of the new profit to old profit for all three products : $\frac{π_{1,1}}{π_{1,0}}$, $\frac{π_{2,1}}{π_{2,0}}$ and $\frac{π_{3,1}}{π_{3,0}}$ and derive the percentage increase or decrease in profit for all three products. Please show the formulae used and the steps in your calculation.***

The base profit of a product i for a base quantity $Q_{i,0}$ sold at a base price $P_{i,0}$ with a retail commission $r_{i}$  is :

$π_{i,0} = Q_{i,0} \cdot (P_{i,0} \cdot (1-r_{i})-C_{i})$

So we deduce that the ratio of profit of product i after versus before the price change is :

$\frac{π_{i,1}}{π_{i,0}} = \frac{Q_{i,1} \cdot (P_{i,1} \cdot (1-r_{i})-C_{i})}{Q_{i,0} \cdot (P_{i,0} \cdot (1-r_{i})-C_{i})}$ 
⇔ $\frac{π_{i,1}}{π_{i,0}} = \frac{Q_{i,1}}{Q_{i,0}} \cdot \frac{(P_{i,1} \cdot (1-r_{i})-C_{i})}{(P_{i,0} \cdot (1-r_{i})-C_{i})}$
⇔ $\frac{π_{1,1}}{π_{1,0}} = \frac{Q_{i,1}}{Q_{i,0}} \cdot \frac{(P_{i,0} \cdot (1+γ_{i}) \cdot (1-r_{i})-C_{i})}{(P_{i,0} \cdot (1-r_{i})-C_{i})}$

So we can compute the profit ratio of each of the three products :

- For the product 1, the ratio between the profit after the price change and before the price change is :

    ⇔ $\frac{π_{1,1}}{π_{1,0}} = \frac{Q_{1,1}}{Q_{1,0}} \cdot \frac{(P_{1,1} \cdot (1-r_{1})-C_{1})}{(P_{1,0} \cdot (1-r_{1})-C_{1})}$

    ⇔ $\frac{π_{1,1}}{π_{1,0}} = \frac{Q_{1,1}}{Q_{1,0}} \cdot \frac{(P_{1,0} \cdot (1+γ_{1}) \cdot (1-r_{1})-C_{1})}{(P_{1,0} \cdot (1-r_{1})-C_{1})}$

    ⇔ $\frac{π_{1,1}}{π_{1,0}} = \frac{Q_{1,1}}{Q_{1,0}} \cdot \frac{(2 \cdot (1+0.05) \cdot (1-0.1)-0.5)}{(2 \cdot (1-0.1)-0.5)}$

    ⇔ $\frac{π_{1,1}}{π_{1,0}} ≈ 0.728 \cdot \frac{1.39}{1.3}$

    ⇔ $\frac{π_{1,1}}{π_{1,0}} ≈ 0.778$

- For the product 2, the ratio between the profit after the price change and before the price change is :

    ⇔ $\frac{π_{2,1}}{π_{2,0}} = \frac{Q_{2,1}}{Q_{2,0}} \cdot \frac{(P_{2,1} \cdot (1-r_{2})-C_{2})}{(P_{2,0} \cdot (1-r_{2})-C_{2})}$

    ⇔ $\frac{π_{2,1}}{π_{2,0}} = \frac{Q_{2,1}}{Q_{2,0}} \cdot \frac{(P_{2,0} \cdot (1+γ_{2}) \cdot (1-r_{2})-C_{2})}{(P_{2,0} \cdot (1-r_{2})-C_{2})}$

    ⇔ $\frac{π_{2,1}}{π_{2,0}} = \frac{Q_{2,1}}{Q_{2,0}} \cdot \frac{(1.75 \cdot (1+0) \cdot (1-0.1)-0.75)}{(1.75 \cdot (1-0.1)-0.75)}$

    ⇔ $\frac{π_{2,1}}{π_{2,0}} ≈ 0.954 \cdot \frac{0.825}{0.825}$

    ⇔ $\frac{π_{2,1}}{π_{2,0}} ≈ 0.954$

- For the product 3, the ratio between the profit after the price change and before the price change is :

    ⇔ $\frac{π_{3,1}}{π_{3,0}} = \frac{Q_{3,1}}{Q_{3,0}} \cdot \frac{(P_{3,1} \cdot (1-r_{3})-C_{3})}{(P_{3,0} \cdot (1-r_{3})-C_{3})}$

    ⇔ $\frac{π_{3,1}}{π_{3,0}} = \frac{Q_{3,1}}{Q_{3,0}} \cdot \frac{(P_{3,0} \cdot (1+γ_{3}) \cdot (1-r_{3})-C_{3})}{(P_{3,0} \cdot (1-r_{3})-C_{3})}$

    ⇔ $\frac{π_{3,1}}{π_{3,0}} = \frac{Q_{3,1}}{Q_{3,0}} \cdot \frac{(1.5 \cdot (1-0.15) \cdot (1-0.1)-0.9)}{(1.5 \cdot (1-0.1)-0.9)}$

    ⇔ $\frac{π_{3,1}}{π_{3,0}} ≈ 1.258 \cdot \frac{0.2475}{0.45}$

    ⇔ $\frac{π_{3,1}}{π_{3,0}} ≈ 0.692$


#### Question B

***Interpret and discuss why each of these changes is happening. Which products benefit from the price changes and why (or why not)? Is product 3’s price cut justified ?***

Product 1: Price Increase

The ratio of the profit after the price change to the profit before the price change is $\frac{π_{1,1}}{π_{1,0}} ≈ 0.778$

- The price increase results in a decrease in demand, as shown by the quantity ratio ($≈ 0.728$) due to the reasons explained in Question 3.
- Despite the drop in demand, the profit per unit increases due to the price increase, but this does not fully offset the demand loss.
- The overall profit decreases to about 78% of the original profit, indicating that the price increase is not beneficial in this case. The demand reduction outweighs the positive impact of the price hike.

Product 2: Price Constant

The ratio of the profit after the price change to the profit before the price change is $\frac{π_{2,1}}{π_{2,0}} ≈ 0.954$

- There is a slight decrease in demand (quantity ratio ≈ 0.954) due to the reasons explained in Question 3.
- Since the price remained constant, the profit margin remains unchanged, and thus the overall profit for product 2 slightly decreases.
- This indicates that product 2 does not benefit or suffer significantly from the price change scenario. The demand is slightly lower, but the constant price ensures the profit margin does not change much.

Product 3: Price Decrease

The ratio of the profit after the price change to the profit before the price change is $\frac{π_{3,1}}{π_{3,0}} ≈ 0.692$

- The price decrease leads to an increase in demand, as shown by the quantity ratio ($≈ 1.258$) due to the reasons explained in Question 3.
- However, the profit margin per unit decreases significantly due to the price reduction.
- The overall profit is reduced to about 69.2% of the original, indicating that the price cut is not justified. The increase in demand is not sufficient to offset the loss in profit per unit, and the overall profit declines.

#### Question C

***Now write a function that takes as input the price change percentages (γ1, γ2, γ3), the price and price elasticities for the three products, the retail margin (r), and the marginal costs C1, C2, C3 and gives as output the ratios of new to old profits: π11 , π21 , π31 . (Note: You can also simplify the function by simply using the vector "quantity_changes" that you stored in Question 3 and thereby avoid using price elasticities in this function. This is not necessary, but you may find that this simplifies your function specification significantly).***

For programming a function that determines the ratio between profits before and after the price change, we have two choices:

- code a function that uses elasticities
- code a function that uses quantity changes

In order to compare these two approaches, we decided to code these two functions and we will then verify that they produce the same result.

The first function for determining profit ratios uses elasticities.

In [9]:
def profit_ratios_with_elasticities(price_changes, elasticities, marginal_costs, base_prices, retail_margin):
    """
    Calculate the profit ratios for a set of products after applying price changes, accounting for elasticities.
    
    Parameters:
    - price_changes (list of float): Percentage changes in prices for each product, where each element
                                     corresponds to the price change for a product (e.g., 0.05 for a 5% increase).
    - elasticities (list of list of float): Cross-elasticities between products, represented as a matrix where
                                            elasticities[i][j] is the elasticity of product i with respect to 
                                            the price of product j.
    - marginal_costs (list of float): The marginal cost associated with each product, representing the variable cost
                                      to produce one unit of each product.
    - base_prices (list of float): Initial (base) prices of each product before any price change is applied.
    - retail_margin (float): The percentage margin that retailers add to the cost price when selling the product.
                             For example, if the retail margin is 0.1, it represents a 10% markup on the product.
                             
    Returns:
    - dict: A dictionary where each key is 'profit_ratio_product_{i+1}' representing the product, and the value is 
            the calculated profit ratio after the price change for that product.
    """
    
    profit_ratios = {}  # Initialize an empty dictionary to store the profit ratios for each product
    num_products = len(base_prices)  # Get the number of products, which is the length of the base_prices list
    
    # Loop through each product to calculate the profit ratios
    for i in range(num_products):
        # Step 1: Calculate the quantity ratio based on the elasticities and price changes
        quantity_ratio = 1  # Initialize quantity ratio for this product to 1 (start value)
        for j in range(num_products):
            # Multiply the current quantity ratio by the effect of the price change for product j
            # This is the elasticity-based adjustment for product i's quantity
            quantity_ratio *= (1 + price_changes[j]) ** elasticities[i][j]
        
        # Step 2: Calculate the base profit (profit before any price changes)
        base_profit = base_prices[i] * (1 - retail_margin) - marginal_costs[i]
        # Step 3: Calculate the new price for product i after applying the price change
        new_price = base_prices[i] * (1 + price_changes[i])
        
        # Step 4: Calculate the profit ratio after the price change for product i
        profit_ratio = quantity_ratio * (new_price * (1 - retail_margin) - marginal_costs[i]) / base_profit
        
        # Step 5: Store the calculated profit ratio in the profit_ratios dictionary
        profit_ratios[f'profit_ratio_product_{i+1}'] = profit_ratio
    
    return profit_ratios  # Return the dictionary containing profit ratios for each product

The second function for determining profit ratios uses quantity changes.

In [10]:
def profit_ratios_with_quantity_changes(price_changes, quantity_changes, marginal_costs, base_prices, retail_margin):
    """
    Calculate the profit ratios for a set of products after applying price changes, 
    accounting for quantity changes due to those price adjustments.
    
    Parameters:
    - price_changes (list of float): Percentage changes in prices for each product, where each element
                                     corresponds to the price change for a product (e.g., 0.05 for a 5% increase).
    - quantity_changes (list of float): Expected change in quantity sold for each product due to price changes.
                                        Each element represents the factor by which quantity changes 
                                        (e.g., 1.1 for a 10% increase, 0.9 for a 10% decrease).
    - marginal_costs (list of float): The marginal cost associated with each product, representing the variable cost
                                      to produce one unit of each product.
    - base_prices (list of float): Initial (base) prices of each product before any price change is applied.
    - retail_margin (float): The percentage margin that retailers add to the cost price when selling the product.
                             For example, if the retail margin is 0.2, it represents a 20% markup on the product.
                             
    Returns:
    - dict: A dictionary where each key is 'profit_ratio_product_{i+1}' representing the product, and the value is 
            the calculated profit ratio after the price and quantity change for that product.
    """
    
    profit_ratios = {}  # Initialize an empty dictionary to store the profit ratios for each product
    num_products = len(base_prices)  # Get the number of products based on the length of base_prices
    
    # Loop through each product to calculate the profit ratios
    for i in range(num_products):
        ratio = 1  # Initialize the profit ratio for the current product as 1 (start value)
        
        # Step 1: Calculate the change in profit for product i using the quantity change
        ratio *= quantity_changes[i] * (base_prices[i] * (1 + price_changes[i]) * (1 - retail_margin) - marginal_costs[i]) / (base_prices[i] * (1 - retail_margin) - marginal_costs[i])
        
        # Step 2: Store the profit ratio for product i in the profit_ratios dictionary
        profit_ratios[f'profit_ratio_product_{i+1}'] = ratio
    
    return profit_ratios  # Return the dictionary containing the profit ratios for each product

#### Question D

***Evaluate the function at the values specified earlier and store the results in an array named "profit_changes". Check if the results from the function are the same as that from in Question A.***

We first start by running and testing the function using the elasticities as input variable.

In [11]:
# List of price changes for each product
price_changes = [0.05, 0, -0.15]  # 5% increase for product 1, no change for product 2, 15% decrease for product 3

# Elasticity matrix representing the responsiveness of product quantity to price changes of all products
elasticities = [
    [-2.5, 1.5, 1.2],  # Elasticities for product 1 in relation to its own price and the prices of products 2 and 3
    [0.7, -1.5, 0.5],  # Elasticities for product 2
    [0.7, 0.4, -1.2]   # Elasticities for product 3
]

# Marginal costs for each product
marginal_costs = [0.5, 0.75, 0.9]  # Marginal cost per unit for product 1, 2, and 3

# Base prices for each product
base_prices = [2, 1.75, 1.5]  # Initial selling price for product 1, 2, and 3

# Retail margin, representing the percentage of the selling price retained after retailer's cut
retail_margin = 0.1  # 10% margin for all products

# Call the profit calculation function using the price changes, elasticities, marginal costs, base prices, and retail margin
profit_ratios_elasticities = profit_ratios_with_elasticities(price_changes, elasticities, marginal_costs, base_prices, retail_margin)

# Output the calculated profit ratios for each product, stored as a dictionary
print(profit_ratios_elasticities)

# Print the type of the object returned by the function (should be a dictionary)
print(type(profit_ratios_elasticities))

{'profit_ratio_product_1': 0.7787551130826625, 'profit_ratio_product_2': 0.95398593595287, 'profit_ratio_product_3': 0.6916597255123126}
<class 'dict'>


To make it easier to read the results of the changes in profits of the different products, we decided to store them in a dictionary. However, for data manipulation reasons we will transform them into a "np.array" format.

In [12]:
# Convert the values of the profit_ratios_elasticities dictionary into a numpy array for easy manipulation
profit_changes_elasticities = np.array(list(profit_ratios_elasticities.values()))

# Print the numpy array containing the profit changes calculated for each product
print(profit_changes_elasticities)

# Print the type of the profit_changes_elasticities object, which should be a numpy array
print(type(profit_changes_elasticities))

[0.77875511 0.95398594 0.69165973]
<class 'numpy.ndarray'>


Then we run and test the function using the quantity changes as input variable.

In [13]:
# List of price changes for each product
price_changes = [0.05, 0, -0.15]  # 5% increase for product 1, no change for product 2, 15% decrease for product 3

# Assuming quantity_changes is already defined (for example, from a previous calculation or function)
quantity_changes = quantity_changes  # Quantity changes for each product after price changes (from previous calculations)

# Marginal costs for each product
marginal_costs = [0.5, 0.75, 0.9]  # Marginal cost per unit for product 1, 2, and 3

# Base prices for each product
base_prices = [2, 1.75, 1.5]  # Initial selling price for product 1, 2, and 3

# Retail margin, representing the percentage of the selling price retained after retailer's cut
retail_margin = 0.1  # 10% margin for all products

# Call the profit calculation function using price changes, quantity changes, marginal costs, base prices, and retail margin
profit_ratios_quantity_changes = profit_ratios_with_quantity_changes(price_changes, quantity_changes, marginal_costs, base_prices, retail_margin)

# Output the calculated profit ratios for each product, stored as a dictionary
print(profit_ratios_quantity_changes)

# Print the type of the profit_ratios_quantity_changes object, which should be a dictionary
print(type(profit_ratios_quantity_changes))

{'profit_ratio_product_1': 0.7787551130826625, 'profit_ratio_product_2': 0.95398593595287, 'profit_ratio_product_3': 0.6916597255123126}
<class 'dict'>


To make it easier to read the results of the changes in profits of the different products, we decided to store them in a dictionary. However, for data manipulation reasons we will transform them into a "np.array" format.

In [14]:
# Convert the values of the profit_ratios_quantity_changes dictionary into a numpy array for easier manipulation
profit_changes_quantity_changes = np.array(list(profit_ratios_quantity_changes.values()))

# Output the numpy array containing the profit changes calculated for each product
print(profit_changes_quantity_changes)

# Print the type of the profit_changes_quantity_changes object, which should be a numpy array
print(type(profit_changes_quantity_changes))

[0.77875511 0.95398594 0.69165973]
<class 'numpy.ndarray'>


To more easily (and scientifically) compare the results of the two functions, we decided to create a function capable of calculating the difference in errors in percentage between two arrays.

In [15]:
# Assign the profit changes calculated using quantity changes to a variable
profit_changes_python = profit_changes_quantity_changes  # Profit changes calculated using quantity changes

# Assign the profit changes calculated using elasticities to a variable
profit_changes_calculator = profit_changes_elasticities  # Profit changes calculated using elasticities

# Calculate the percentage errors between the two profit change calculations
errors = calculate_percentage_errors(profit_changes_python, profit_changes_calculator)

# Output the calculated errors as a list of percentage differences
print(errors)

[0.0, 0.0, 0.0]


As we can see the difference in errors between the two functions for each products is 0%, so we can conclude that the two functions provide the same results. However, for reasons of ease and consistency in our approach, we recommend choosing the function that uses quantity changes as an input variable.

Now we want to compare the results of the function programmed in python (with the changes in quantities) and the calculations performed with the calculator.

In [16]:
# Assign the profit changes calculated using quantity changes to a variable
profit_changes_python = profit_changes_quantity_changes  # Profit changes calculated using quantity changes

# Define a numpy array for the actual profit changes (calculated by hand)
profit_changes_calculator = np.array([0.778, 0.954, 0.692])  # Actual profit changes for comparison

# Calculate the percentage errors between the two profit change calculations
errors = calculate_percentage_errors(profit_changes_python, profit_changes_calculator)

# Output the calculated errors as a list of percentage differences
print(errors)

[0.09696412517581306, 0.00147424051026023, 0.04919680518268182]


As we can see the difference in errors between the two methods for :

- the product 1 is approximatively 0.1%
- the product 2 is approximatively 0.001%
- the product 3 is approximatively 0.05%

We can therefore conclude with confidence that the results between the two methods are substantially identical. The function gives us the same results as the calculations.

### Question 5

***Can you calculate the highest marginal cost at which this price cut is justified for product 3?***

In the current situation, the marginal cost of product 3 is $0.9 and it suffers a price cut of 15% (i.e. 0.15). However, with a profit decrease of about 31%, the price cut is not justified given the current marginal cost of product 3.

*Reminder* : the ratio of the profit after the price change to the profit before the price change is $\frac{π_{3,1}}{π_{3,0}} ≈ 0.692$

We can define a price cut as being justified if it allows for increased profits, that is to say that the ratio between the profits after and before the price cut is: $\frac{π_{3,1}}{π_{3,0}} > 1$

So, to determine what is the highest marginal cost that justifies the price cut for product 3, we will solve the following equation with unknown $C_{3}$:

$\frac{Q_{3,1}}{Q_{3,0}} \cdot \frac{(P_{3,0} \cdot (1+γ_{3}) \cdot (1-r_{3})-C_{3})}{(P_{3,0} \cdot (1-r_{3})-C_{3})} = 1$

In [17]:
def solve_for_highest_marginal_cost(base_price, quantity_change, price_change, retail_margin):
    """
    This function solves for the marginal cost (MC) that maximizes a certain expression.

    Parameters:
    base_price (float): The base price of the product.
    quantity_change (float): The change in quantity of the product.
    price_change (float): The change in price of the product.
    retail_margin (float): The retail margin of the product.

    Returns:
    C3_solution (float): The value of the marginal cost that maximizes the expression.
    """
    # Create a symbolic variable for marginal cost
    MC = symbols('MC')

    # Define the left-hand side of the equation
    lhs = quantity_change * (base_price * (1 + price_change) * (1 - retail_margin) - MC)

    # Define the right-hand side of the equation
    rhs = base_price * (1 - retail_margin) - MC

    # Create an equation by setting the left-hand side equal to the right-hand side
    equation = Eq(lhs / rhs, 1)

    # Solve the equation for the marginal cost
    C3_solution = solve(equation, MC)

    # Return the solution
    return C3_solution

In [18]:
# Set the base price of the product
base_price = 1.5

# Set the quantity change of the product to the third element of the quantity_changes array
quantity_change = quantity_changes[2]

# Set the price change of the product to a decrease of 15%
price_change = -0.15

# Set the retail margin of the product to 10%
retail_margin = 0.1

# Call the solve_for_highest_marginal_cost function with the specified parameters
C3_solution = solve_for_highest_marginal_cost(base_price, quantity_change, price_change, retail_margin)

# Print the solution to the console
print(C3_solution)

# Print the type of the solution to the console
print(type(C3_solution))

[0.361285007720345]
<class 'list'>


The highest marginal cost at which the cut price is justified for product 3 is $C_{3} ≈ 0.36$ because that this is the maximum value of marginal cost at which the profit ratio is equal to 1 (meaning that the price change has not changed profits).